[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/repos-especializacion-UdeA/monografia_modelos/blob/main/new_notebook.ipynb)

In [1]:
try:
    import google.colab 
    !wget --no-cache -O init.py -q https://raw.githubusercontent.com/repos-especializacion-UdeA/monografia_modelos/refs/heads/main/init.py
    from init import init; init(force_download=False)
except ImportError:
    print("Ejecucion del notebook en entorno local")

Ejecucion del notebook en entorno local


## 1. Librerias y configuraciones previas

In [2]:
# Verificacion de librerias necesarias antes de empezar
try:
    import scipy.io
    print("La librería 'scipy' está instalada y se ha importado correctamente.")
except ImportError:
    print("La librería 'scipy' no está instalada.")
    print("Instalando 'scipy'...")
    !pip install scipy

try:
    import libemg
    print("La librería 'libEMG' está instalada y se ha importado correctamente.")
except ImportError:
    print("La librería 'libEMG' no está instalada.")
    print("Instalando 'libEMG'...")
    !pip install libemg

try:
    import imblearn
    print("La librería 'imblearn' está instalada y se ha importado correctamente.")
except ImportError:
    print("La librería 'imblearn' no está instalada.")
    print("Instalando 'imblearn'...")
    !pip install imblearn

try:
    import mlflow
    print("La librería 'mlflow' está instalada y se ha importado correctamente.")
except ImportError:
    print("La librería 'mlflow' no está instalada.")
    print("Instalando 'mlflow'...")
    !pip install mlflow

La librería 'scipy' está instalada y se ha importado correctamente.
La librería 'libEMG' está instalada y se ha importado correctamente.
La librería 'imblearn' está instalada y se ha importado correctamente.
La librería 'mlflow' está instalada y se ha importado correctamente.


In [15]:
import os
import re
import pandas as pd

* jsonfile = open("local/data/emp1.json", "w")
* jsonfile.write(emp1_str)
* jsonfile.close()

In [20]:
def load_and_merge_csv(directory_path):
    """
    Carga todos los archivos CSV de un directorio, extrae un número identificador del nombre de archivo
    y concatena todos los datos en un único DataFrame con una columna adicional 'file_id'.

    Parámetros:
        directory_path (str): Ruta al directorio que contiene los archivos CSV.

    Retorna:
        pd.DataFrame: DataFrame con los datos combinados y columna 'file_id'.

    Requisitos:
        Los arhivos tienen la forma subject_1_features.csv, subject_2_features.csv, etc.
    """
    all_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    merged_data = []

    for file in all_files:
        file_path = os.path.join(directory_path, file)
        df = pd.read_csv(file_path)

        # Extrae el primer número del nombre del archivo (puedes ajustar la expresión regular si es necesario)
        match = re.search(r'\d+', file)
        file_id = int(match.group()) if match else None

        df['subject'] = file_id
        merged_data.append(df)

    return pd.concat(merged_data, ignore_index=True)

In [21]:
# Specify the directory path
directory_path = './local/data'

# Load and merge CSV files
merged_df = load_and_merge_csv(directory_path)

In [22]:
# Display the first few rows of the merged DataFrame
merged_df.shape

(21594, 42)

In [23]:
# Display the first few rows of the merged DataFrame
merged_df.head()

,RMS_f0,RMS_f1,RMS_f2,RMS_f3,RMS_f4,RMS_f5,RMS_f6,RMS_f7,RMS_f8,RMS_f9,...,SSC_f2,SSC_f3,SSC_f4,SSC_f5,SSC_f6,SSC_f7,SSC_f8,SSC_f9,restimulus,subject
0,0.186650,0.064954,0.007956,0.002370,0.002316,0.002340,0.011819,0.025321,0.002160,0.021807,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,10
1,0.147451,0.024014,0.010991,0.002311,0.002293,0.003349,0.061164,0.166142,0.001954,0.076644,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,10
2,0.214203,0.027382,0.070140,0.002309,0.002581,0.005600,0.146619,0.526502,0.002533,0.311813,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,10
3,0.399017,0.076350,0.197038,0.002660,0.003711,0.007535,0.219384,0.941561,0.005587,0.659597,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10
4,0.664900,0.140433,0.336471,0.003742,0.006055,0.008054,0.247355,1.144152,0.011067,0.949690,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,10


In [32]:
def split_features_by_type(df):
    """
    Divide un DataFrame en un diccionario donde las claves son las características
    encontradas en los nombres de las columnas y los valores son DataFrames
    con las columnas correspondientes a esa característica, incluyendo las columnas
    'restimulus' y 'subject'.

    Parámetros:
        df (pd.DataFrame): DataFrame con todas las características.

    Retorna:
        dict: Diccionario con claves como nombres de características (e.g., 'RMS', 'MAV')
              y valores como DataFrames con las columnas correspondientes.
    """
    feature_dict = {}
    for column in df.columns:
        # Extraer el prefijo de la característica (e.g., 'RMS', 'MAV', etc.)
        match = re.match(r'([A-Z]+)_', column)
        if match:
            feature_type = match.group(1)
            if feature_type not in feature_dict:
                feature_dict[feature_type] = []
            feature_dict[feature_type].append(column)
    
    # Convertir las listas de columnas en DataFrames e incluir 'restimulus' y 'subject'
    for feature_type in feature_dict:
        feature_dict[feature_type] = df[feature_dict[feature_type] + ['restimulus', 'subject']]
    
    return feature_dict

In [33]:
# Dividir las características por tipo
features_dict = split_features_by_type(merged_df)

In [34]:
features_dict.keys()

dict_keys(['RMS', 'MAV', 'ZC', 'SSC'])

In [31]:
features_dict['RMS'].head()

,RMS_f0,RMS_f1,RMS_f2,RMS_f3,RMS_f4,RMS_f5,RMS_f6,RMS_f7,RMS_f8,RMS_f9,restimulus,subject
0,0.186650,0.064954,0.007956,0.002370,0.002316,0.002340,0.011819,0.025321,0.002160,0.021807,0.0,10
1,0.147451,0.024014,0.010991,0.002311,0.002293,0.003349,0.061164,0.166142,0.001954,0.076644,0.0,10
2,0.214203,0.027382,0.070140,0.002309,0.002581,0.005600,0.146619,0.526502,0.002533,0.311813,-1.0,10
3,0.399017,0.076350,0.197038,0.002660,0.003711,0.007535,0.219384,0.941561,0.005587,0.659597,1.0,10
4,0.664900,0.140433,0.336471,0.003742,0.006055,0.008054,0.247355,1.144152,0.011067,0.949690,1.0,10


In [35]:
features_dict['MAV'].head()

,MAV_f0,MAV_f1,MAV_f2,MAV_f3,MAV_f4,MAV_f5,MAV_f6,MAV_f7,MAV_f8,MAV_f9,restimulus,subject
0,0.183693,0.060620,0.007765,0.002369,0.002316,0.002333,0.008609,0.023143,0.002157,0.020405,0.0,10
1,0.147227,0.021864,0.009099,0.002311,0.002293,0.003275,0.052383,0.127750,0.001953,0.055653,0.0,10
2,0.207631,0.024446,0.057055,0.002309,0.002569,0.005478,0.140409,0.486567,0.002465,0.277829,-1.0,10
3,0.385928,0.071161,0.184839,0.002645,0.003640,0.007516,0.218086,0.926799,0.005241,0.639306,1.0,10
4,0.652486,0.137464,0.332023,0.003689,0.005926,0.008054,0.247321,1.143647,0.010746,0.944538,1.0,10


## Data frame Nuevo

### Pendiente

1. Generar DF completo
2. Drop a las features no usadas. Si vamos usar RMS, drop a MAV; ZC, SSC
3. * df_RMS Subject: rms_channel1, rms_channel2, ..., rms_channel3, grasp_label
     * Boxplots por canal
4. train-test split (80 - 20)
5. model fit:
   * SVM
   * KNN
   * CNN (Ultima a emplear)
   * RF 
6. Compare metrics:
   * Accuracy
   * ROC
   * F1 score
   * F2 score 

**Objetivo**: 
1. Hacer esto con MAV y RMS
2. Todas menos CNN


